In [ ]:
#test the geo distance matrix, added in the function files
#weighted cluster method
library('data.table')
library('ggplot2')
library('ggmap')
library('dbscan')
library('dplyr')
library('xts')
#smu
funPath='D:/share/Git/Rprojects/ECA/function.R'

#funPath='D:/Git/Rprojects/ECA/function.R'
source(funPath)

In [ ]:
#get clusters
findCluster<-function(dt,eps=0.002,minp=10){
    
    temp1 <- as.matrix(dt[,list(lon,lat)])
    cl <- dbscan(temp1, eps = eps, minPts =minp)
    temp2=data.table(cbind(dt,c=cl$cluster));  
    return(temp2[c>0])
    
}

In [ ]:
#extract each trip from each cluster and mmsi
#input:candidate clusters
#output:cluster and duration for each ship in hours
findTrip<-function(dt,eps=6000*2,minp=5){
    
    temp3=dt[,.N,list(c,mmsi)]
    nr=nrow(temp3)#how many ships
    #nr=14
    tt=dt[mmsi<0];tt=tt[,c1:=0][c1<0]
    t0=data.table(mmsi=0,c=0,c1=0,dur=0)[mmsi<0]
    for (i in seq(1:nr)){
    r=temp3[i]
    cship=dt[mmsi==r$mmsi&c==r$c]
    setkey(cship,mmsi,time)
    cship2 <- as.matrix(cship[,list(time)])#get time series
    cl2 <- dbscan(cship2, eps = eps, minPts =minp)
    cship3=data.table(cbind(cship,c1=cl2$cluster));
    tt=rbind(tt,cship3)
    #if(nrow(cship3[c1>0])>0){
    #    t1=cship3[c1>0,list(dur=(max(.SD$time)-min(.SD$time))/3600),by=list(c,mmsi,c1)]
    #    t0=rbind(t0,t1) 
    #}
    }
    return(tt)

}


In [ ]:
#input eps is for dbscan to combine different clusters which actually at the same terminal but seperate by two or more grids
#eps set to be the doubel of the eps used to identify candidates terminals
#minp set to 2 as we only sample one point from each candidate cluster for combination purpose
getFinalCluster<-function(dt,eps=0.002*2,minp=2){
    sampled=dt[,.SD[median(1:.N)],list(glon,glat,c)]# the points produced with medium are representative
    cl=sampled[,.N,list(glon,glat,c)]
    cl=cbind(cl,cid=seq(1:nrow(cl)))#set new cluster ids
    
    sampled1=data.table(left_join(sampled,cl[,list(glon,glat,c,cid)],by=c('glon','glat','c')))
    cls=findCluster(sampled1[,list(c.org=c,glon,glat,lon,lat,cid)],eps,minp)#doubel the eps
    s2=cls[c>0,.N,list(glon,glat,c,c.org,cid)];
    s3=s2[,N,c];
    dt1=data.table(left_join(dt,cl[,list(glon,glat,c,cid)],by=c('glon','glat','c')))
    dt1=dt1[,fcid:=cid]#add final cluster identification
    #replace the cluster id at the same terminal
   
    for (i in unique(s2$c)){
        cids=s2[c==i]$cid
        dt1=dt1[fcid%in%cids,fcid:=min(cids)]  
    }
    return(dt1)    
}

In [ ]:
Sys.time()
dt=data.table(mmsi=0,time=0,status=0,sog=0,lon=0,lat=0)[mmsi<0]
path='D:/share/AIS/globalContainer2015/'
for(i in seq(201501,201512)){
    
    filepath=paste(path,i,sep='')
    temp=fread(filepath)[,list(mmsi,time,status,sog,lon,lat)];
    dt=rbind(dt,temp)
    }
Sys.time()

In [ ]:
setkey(dt,mmsi,time)

In [ ]:
dt0=dt[status==5&sog==0];head(dt0);dim(dt0);setkey(dt0,mmsi,time)

In [ ]:
nrow(dt0[,.N,mmsi])
nrow(candidates[,.N,mmsi])
nrow(selected[,.N,mmsi])

In [ ]:
#sample 
tmp=dt0[,atime:=align.time(as.POSIXct(time,origin='1970-01-01'),60*20)];dim(tmp);head(tmp);nrow(tmp[,.N,list(mmsi,atime)])
d5=tmp[,.SD[1],list(mmsi,atime)];dim(d5);head(d5)

In [ ]:
#set grids
scale=2
d5=d5[,glon:=floor(lon*scale)/scale]
d5=d5[,glat:=floor(lat*scale)/scale]
d6=d5[,.N,list(glon,glat)]
setorder(d6,-N)
head(d6);nrow(d6)

In [ ]:
#combine results of all grids
st=Sys.time()
eps=0.002
minp=10
candidates=d5[mmsi<0];
candidates=candidates[,c:=0]
candidates=candidates[,c1:=0][c<0]
for (i in seq(1,nrow(d6))){
    print(i)
    r=d6[i]    
    temp=d5[glon==r$glon&glat==r$glat]
    temp2=findCluster(temp,eps,minp)
    if(nrow(temp2)>0){
        
    t0=findTrip(temp2)
    candidates=rbind(candidates,t0)
        
    }
    
}
Sys.time()-st

In [ ]:
head(candidates);dim(candidates[,.N,list(glon,glat,c)]);dim(candidates)

In [ ]:
nrow(candidates[,.N,list(glon,glat,c)])
nrow(candidates2[,.N,list(fcid)])

In [ ]:
#add unique cluster cid and combine clusters belong to the same termainal
candidates2=getFinalCluster(candidates,0.004,2)#just use the selected replace the tempsample, here is only for test purpose
dim(candidates2);head(candidates2)
#head(sampled);dim(sampled)

In [ ]:
#number of trips with duration 168462
t1=candidates2[c>0&c1>0,list(dur=(max(.SD$time)-min(.SD$time))/3600),by=list(glon,glat,c,mmsi,c1,fcid)]
head(t1);dim(t1);

In [ ]:
hist(t1[fcid==676]$dur,breaks=100)
hist(t1[fcid==690]$dur,breaks=100)
hist(t1[fcid==2219]$dur,breaks=100)
hist(t1[fcid==1]$dur,breaks=100)

In [ ]:
#head(t0)
t2=t1[c>0&c1>0,list(uniques=length(unique(.SD$mmsi)),mean=mean(dur),median=median(dur),sd=sd(dur),.N),list(fcid)]
#t3=t2[,norm:=abs(mean-median)/max(mean,median)]#incorrect
max=apply(t2[,list(mean,median)],1,max)
t3=cbind(t2,max)
t3=t3[,norm:=abs(mean-median)/max]
t3=t3[,tripRate:=N/uniques]
t4=t3[norm<0.25&uniques>5]

dim(t4);t3[fcid%in%c(676,678,679,690,691)]

In [ ]:
sampled=candidates2[c>0&c1>0,sample_n(.SD,5),fcid]
head(sampled);dim(sampled)
write.csv(sampled,'d:/share/terminals.csv')
write.csv(t3,'d:/share/t3.csv')

In [ ]:
min(candidates2[c>0&c1>0,.N,fcid]$N)
nrow(sampled[,.N,fcid])
nrow(candidates2[c>0&c1>0,.N,fcid])

In [ ]:
selected=data.table(inner_join(candidates2,t4[,list(fcid)],by='fcid'))
dim(selected);head(selected);

In [ ]:
zoomSize=7
#temp=candidates2[lon>119&lon<124&lat<31.5&lat>29,.SD[1],list(fcid)];nrow(temp)
temp=selected[lon>117&lon<124&lat<35&lat>29,.SD[1],list(fcid)];nrow(temp)

#temp=selected[,.SD[1],list(fcid)];nrow(temp)
centerX=0.5*(max(temp$lon)+min(temp$lon))
centerY=0.5*(max(temp$lat)+min(temp$lat))
#p<-ggmap(get_map(location=c(centerX,centerY),zoom=zoomSize,source='google',maptype = 'satellite'))
p=ggplot()
p=p+geom_point(data=temp,aes(x=lon,y=lat,col=as.factor(fcid)),size=2,alpha=0.75)
p=p+geom_text(data=temp[,.SD[1],fcid],nudge_x = 0.05,nudge_y = 0.05,aes(x=lon,y=lat,label=fcid),color='black',size=2)
p=p+labs(x="Longtitude",y="Latitude")+theme(legend.position='none')
#p
#hulls=temp[chull(temp[,list(lon,lat)]),]
#p=p+geom_point(data = hulls, aes(x = lon, y = lat),size=1,col='yellow') 
#p=p+geom_polygon(data = hulls, aes(x = lon, y = lat),fill='yellow',alpha = 0.4)
p

In [ ]:
#draw each cluster
tc=selected[,.N,fcid]
tc=tc[1:3]
for (i in seq(1:nrow(tc))){
    #zoomSize=8
    r=tc[i]
    temp=selected[glon==r$glon&glat==r$glat&c==r$c];nrow(temp)
    #centerX=0.5*(max(temp$lon)+min(temp$lon))
    #centerY=0.5*(max(temp$lat)+min(temp$lat))
    #p<-ggmap(get_map(location=c(centerX,centerY),zoom=zoomSize,source='google',maptype = 'satellite'))
    p=ggplot()
    p=p+geom_point(data=temp,aes(x=lon,y=lat,col=as.factor(paste(glon,glat,c))),size=2,alpha=0.75)
    p=p+geom_text(data=temp[,.SD[1],list(glon,glat,c)],nudge_x = 0.005,nudge_y = 0.005,aes(x=lon,y=lat,label=c),color='black',size=4)
    p=p+labs(x="Longtitude",y="Latitude")+theme(legend.position=c(0.9,0.9))
    #p
    hulls=temp[chull(temp[,list(lon,lat)]),]
    p=p+geom_point(data = hulls, aes(x = lon, y = lat),size=1,col='yellow') 
    p=p+geom_polygon(data = hulls, aes(x = lon, y = lat),fill='yellow',alpha = 0.4)
    plot(p) 
    
}

In [ ]:
zoomSize=2
#temp=selected[lon>117&lon<124&lat<35&lat>29,.SD[1],list(fcid)];nrow(temp)

temp=candidates2[,.SD[1],list(fcid)][lon>(-130)&lon<(-120)&lat<50&lat>35];nrow(temp)
centerX=0.5*(max(temp$lon)+min(temp$lon))
centerY=0.5*(max(temp$lat)+min(temp$lat))
#p<-ggmap(get_map(location=c(centerX,centerY),zoom=zoomSize,source='google',maptype = 'satellite'))
p=ggplot()
p=p+geom_point(data=temp,aes(x=lon,y=lat,col=as.factor(fcid)),size=2,alpha=0.75)
p=p+geom_text(data=temp[,.SD[1],fcid],nudge_x = 0.05,nudge_y = 0.05,aes(x=lon,y=lat,label=fcid),color='black',size=2)
p=p+labs(x="Longtitude",y="Latitude")+theme(legend.position='none')
#p
#hulls=temp[chull(temp[,list(lon,lat)]),]
#p=p+geom_point(data = hulls, aes(x = lon, y = lat),size=1,col='yellow') 
#p=p+geom_polygon(data = hulls, aes(x = lon, y = lat),fill='yellow',alpha = 0.4)
p

In [ ]:
fcids=candidates2[lon>(-130)&lon<(-120)&lat<50&lat>42.5,.N,fcid]$fcid
fcids
t3[fcid%in%fcids]

In [ ]:
t=selected[c>0&c1>0,list(time=.SD[1]$time,.N),list(mmsi,fcid,cid,c1)];setkey(t,mmsi,time,fcid)

In [ ]:
t[mmsi==209006000];dim(t)

In [ ]:
aship=dt[mmsi==209006000];setkey(aship,mmsi,time)
dim(aship)

In [ ]:
plot(aship$lon,aship$lat)
plot(aship[sog<25]$sog)

In [ ]:
#start with networks
#read labelled terminals and world port
tm0=fread('D:/share/Git/Rprojects/ShipEmissions/data/terminals_labelled.csv')
port0=fread('D:/share/Git/Rprojects/ShipEmissions/data/worldport.csv')

In [ ]:
dim(tm0);dim(port0);dim(tm0[label==1&uniques>4&N>19])

In [ ]:
#get ships pass the selected terminals
#order trips of each ship based on time
#find the top1 OD in terms of popularity over the network
fcids=unique(tm0[label==1&uniques>4&N>19]$fcid);length(fcids);
sdt=candidates2[fcid%in%fcids];dim(sdt);head(sdt);setkey(sdt,mmsi,c,c1,cid,fcid,time)

In [ ]:
trips=sdt[c>0&c1>0,.SD[c(1,nrow(.SD))],list(glon,glat,c,mmsi,c1,fcid)];nrow(trips)

In [ ]:
trips1=trips[,list(stime=.SD[1]$time,etime=.SD[nrow(.SD)]$time,satime=.SD[1]$atime,eatime=.SD[nrow(.SD)]$atime),list(glon,glat,c,mmsi,c1,fcid)];nrow(trips)

In [ ]:
dim(trips1);head(trips1);setkey(trips1,mmsi,stime);trips1=cbind(trips1,tpid=seq(1,nrow(trips1)));head(trips1)

In [ ]:
nr=nrow(trips1)
ln1=trips1[1:(nr-1),list(mmsi1=mmsi,tpid1=tpid,stime1=stime,etime1=etime,satime1=satime,eatime1=eatime,fcid1=fcid,glon1=glon,glat1=glat)]
ln2=trips1[2:nr,list(mmsi2=mmsi,tpid2=tpid,stime2=stime,etime2=etime,satime2=satime,eatime2=eatime,fcid2=fcid,glon2=glon,glat2=glat)]
ln=cbind(ln1,ln2);dim(ln);ln=ln[mmsi1==mmsi2];dim(ln);head(ln)


In [ ]:
#need to look at the average speed and tim
od=ln[,.N,list(fcid1,fcid2)];dim(od);setorder(od,-N);head(od[fcid1!=fcid2])

In [ ]:
#convert the csv document to esri shp documents
#must have a lon and lat column as longitude and latitude
#infile="D:/Git/data/terminals.csv"
data2shp<-function(MyData,filepath){ 
    library(rgdal)
    library('sp')
    coordinates(MyData)<-~lon+lat # whatever the equivalent is in your 
    writeOGR(MyData, filepath, "layer name", driver = "ESRI Shapefile")  
}


In [ ]:

infile="D:/share/Git/Rprojects/ShipEmissions/data/terminals_labelled.csv"
portfile1="D:/share/Git/Rprojects/ShipEmissions/data/worldport.csv"
portfile2="D:/share/Git/ports.csv" #port data from bloomoo
outfile="D:/share/Git/Rprojects/ShipEmissions/data/terminals.shp"
terminals=fread(infile);terminals=terminals[,V1:=NULL];head(terminals,1);

ports1=fread(portfile1)
ports2=fread(portfile2,sep='@');head(ports2,1)

In [ ]:
data2shp(ports1,"D:/share/Git/Rprojects/ShipEmissions/data/ports_wpi.shp")
data2shp(ports2,"D:/share/Git/Rprojects/ShipEmissions/data/ports_blm.shp")
data2shp(terminals,"D:/share/Git/Rprojects/ShipEmissions/data/terminals.shp")

In [ ]:
#create shp file for selected terminals
fcids=sdt[,.N,fcid]$fcid;selected_terminals=sdt[fcid%in%fcids,.SD[median(seq(1,nrow(.SD)))],fcid];
nrow(selected_terminals);head(selected_terminals);
data2shp(selected_terminals,"D:/share/Git/Rprojects/ShipEmissions/data/selected_terminals.shp")

In [ ]:
#get trip trajectories between fcid 314 and fcid 140
head(ln,1)

In [ ]:
aship=dt[mmsi==209006000]

In [ ]:
r0=data.table(dt[1],fcid1=0,fcid2=0,tpid=0,dur=0)[mmsi<0];head(r0)
n=nrow(ln[fcid1==314&fcid2==140])


In [ ]:
#dt : the original AIS records
#ln:trip lines with start and end terminal
#result will not only include the original AIS record columns 
#but also added start and end terminal, but also duration (s) and tripid(as tpid)
gettripdt<-function(dt,ln){
    n=nrow(ln)
    r0=data.table(dt[1],fcid1=0,fcid2=0,tpid=0,dur=0)[mmsi<0];head(r0)
     
    for(i in seq(1,n)){
        l=ln[i]
        print(i)
        if(l$etime1<l$stime2){
            temp=dt[mmsi==l$mmsi1]
            temp=temp[time>=l$etime1]
            temp=temp[time<=l$stime2]
            temp=temp[,fcid1:=l$fcid1]
            temp=temp[,fcid2:=l$fcid2]
            temp=temp[,tpid:=l$tpid1]
            temp=temp[,dur:=(l$stime2-l$etime1)]
            r0=rbind(r0,temp)
        }
     
    }
    return(r0)
}

In [ ]:
templn=ln[fcid1==314&fcid2==140];
tripdt=gettripdt(dt,templn);

In [ ]:
head(tripdt);dim(tripdt)

In [ ]:
temp=ln[fcid1==314&fcid2==140];temp=temp[,dur:=(stime2-etime1)/3600];hist(log2(temp$dur),breaks=40)

In [ ]:
getTripLine<-function(tripdt){
    r0=data.table(mmsi=0,tpid=0,fcid1=0,fcid2=0,status1=0,time1=0,sog1=0,lon1=0,lat1=0,time2=0,status2=0,sog2=0,lon2=0,lat2=0,dist=0,dur=0)[mmsi<0]
    setkey(tripdt,mmsi,tpid,time)
    tps=tripdt[,.N,tpid]
    n=nrow(tps)
   
    for(i in seq(1,n)){
        id=tps[i]$tpid
        atrip=tripdt[tpid==id]
        m=nrow(atrip)
        if(m>1){
            atrip1=atrip[1:(m-1),list(mmsi,tpid,fcid1,fcid2,status1=status,time1=time,sog1=sog,lon1=lon,lat1=lat)]
            atrip2=atrip[2:m,list(time2=time,status2=status,sog2=sog,lon2=lon,lat2=lat)]
            tripln=cbind(atrip1,atrip2)
            temp=tripln[,dist:=distance(lon1,lat1,lon2,lat2)]
            temp=temp[,dur:=abs(time2-time1)]
            r0=rbind(r0,temp)
        }
        
    }

    return(r0)
    
}

In [ ]:
tripln=getTripLine(tripdt)

In [ ]:
dim(tripln);head(tripln);sum(tripln$dist);sum(tripln$dur)

In [ ]:
temp=tripln[,list(.N,sum(dist),sum(dur)),tpid];head(temp);hist(log(temp$V2/1000),breaks=100);plot(temp$V3/3600,temp$V2/1852)

In [ ]:
hist(temp[V2<3600*24*7]$V2/1852,breaks=30)

In [ ]:
temp2=candidates2[c>0&c1>0,.SD[1],fcid];head(temp2);dim(temp2)

In [ ]:
terminals1=data.table(left_join(temp2,terminals,'fcid'));head(temp3);dim(temp3)

In [ ]:
dim(tripln)

In [ ]:
write.csv(tripln,"D:/share/Git/Rprojects/ShipEmissions/data/tripln314_140.csv")